Let aussume that we need to make this computation

$\sqrt{(a+b)\times c}$

and we want to compute the standard deviation of the result assuming that a, b and c are normal distributed independent variables. Clearly the problem is quite naive but we want to address is as if we will need a cluster to solve it. 

We can partition the problem in a three conscutive operations
1. A sum: $(a+b)$
2. A multiplication of the result 1 with c: $(a+b)\times c$
3. A sqrt of the result of 2: $\sqrt{(a+b)\times c}$

In [1]:
import anytree    # pip install anytree 
import os          
from shutil import copytree
import ruamel.yaml # pip install ruamel.yaml 
import yaml        # pip install pyyaml 
from anytree import AnyNode
from anytree.exporter import DictExporter
from anytree.importer import DictImporter

from anytree import AnyNode, NodeMixin, RenderTree
class NodeJobBase(object):  # Just an example of a base class
    name = None
    path = None
    def __str__(self):
            return self.name

class NodeJob(NodeJobBase, NodeMixin):  # Add Node feature
    def __init__(self, parent=None, children=None, name= None, parent_path=None,
                 path=None,
                 template_path=None, dictionary = None):
        super(NodeJobBase, self).__init__()
        self.name = name
        self.parent = parent
        self.path = path
        self.template_path = template_path
        self.parent_path = parent_path
        self.dictionary= dictionary
        
        if children:  # set children only if given
            self.children = children
    
    def print_it(self):
        for pre, _, node in RenderTree(self, style=anytree.render.ContRoundStyle()):
            print(f"{pre}{node.name}")
    
    def run(self):
        os.system(f'cd {self.path}; python run.py;')
        
    def clone_children(self):
        for child in self.children:
            copytree(child.template_path, child.path)
            child.save_tree()
    
    def rm_children(self,):
        for child in self.children:
            os.system(f'rm -rf {child.path}')
                        
    def mutate(self):
        #https://stackoverflow.com/questions/7255885/save-dump-a-yaml-file-with-comments-in-pyyaml
        ryaml = ruamel.yaml.YAML()
        
        with open(self.path+'/config.yaml', 'r') as file:
            cfg = ryaml.load(file)
        for ii in self.dictionary.keys():
            cfg[ii]=self.dictionary[ii]
    
        with open(self.path+'/config.yaml', 'w') as file:
            ryaml.dump(cfg, file)
                        
    def mutate_children(self):
        for child in self.children:
            child.mutate()
    
    def save_tree(self, filename='tree.yaml'): 
        with open(f"{self.path}/{filename}", "w") as file:  
            yaml.dump(DictExporter().export(self), file)
   
    def generation(self, number):
        return [ii for ii in anytree.search.findall(self, 
                filter_=lambda node: node.depth==number)]

def load_tree(filename='tree.yaml'): 
        with open("tree.yaml", "r") as file:
            return DictImporter(nodecls=NodeJob).import_(yaml.load(file, Loader=yaml.FullLoader))

In [71]:
a=np.random.randn(10)
b=np.random.randn(10)
c=np.random.randn(3)

my_list=[]
for ii in c:
    my_list+=list(np.sqrt(np.abs((a+b)*ii)))
my_list

[0.25132301513700156,
 0.39144959984662053,
 0.5544650918251962,
 0.7093722370273928,
 0.6745889558069091,
 0.9069304602168949,
 0.35901349289035733,
 0.14584741328859446,
 0.9971864303597924,
 1.234763958133191,
 0.3912864918336797,
 0.6094505135957652,
 0.8632504289599321,
 1.1044264047180474,
 1.0502720803486372,
 1.4120061305246014,
 0.5589505206176463,
 0.22707081823605318,
 1.5525262572030443,
 1.9224123073536714,
 0.23671105413975146,
 0.3686906564118938,
 0.5222284011597194,
 0.6681292197321962,
 0.6353682441420704,
 0.8542013756772899,
 0.3381403899923168,
 0.1373678209465997,
 0.9392098490289441,
 1.1629745806771825]

In [53]:
#root
root = NodeJob(name='root', parent=None)
root.path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree'

#first generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child}",
                           parent=node,
                           path = f"{node.path}/{child}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/sum_it',
                           dictionary={'a':float(a[child]), 'b':float(b[child])})
                   for child in range(len(a))]

#second generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child}",
                           parent=node,
                           path = f"{node.path}/{child}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/multiply_it',
                           dictionary={'c':float(c[child])})
                   for child in range(len(c))]
    
#third generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child}",
                           parent=node, 
                           path = f"{node.path}/{child}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/square_root_it',
                           dictionary={'a':float(c[child])})
                           for child in range(1)]

In [54]:
root.print_it()

root
├── 0
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 1
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 2
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 3
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 4
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 5
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 6
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 7
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 8
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
╰── 9
    ├── 0
    │   ╰── 0
    ├── 1
    │   ╰── 0
    ╰── 2
        ╰── 0


In [55]:
import numpy as np
root.path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree'

# sum
for ii, node in enumerate(root.generation(1)): 
    node.path = node.parent.path + '/' + node.name
    node.template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/sum_it'
    node.dictionary={'a':float(a[ii]), 'b': float(b[ii])}
    
# product
for ii, node in enumerate(root.generation(2)): 
    node.path = node.parent.path + '/' + node.name
    node.template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/multiply_it'

# sqrt
for node in root.generation(3): 
    node.path = node.parent.path + '/' + node.name
    node.template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/square_root_it'
    node.dictionary={'a':float(0)}

In [56]:
# save teh tree
root.save_tree()

In [57]:
# Load the tree from a yaml
root=load_tree()
root.print_it()

root
├── 0
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 1
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 2
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 3
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 4
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 5
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 6
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 7
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
├── 8
│   ├── 0
│   │   ╰── 0
│   ├── 1
│   │   ╰── 0
│   ╰── 2
│       ╰── 0
╰── 9
    ├── 0
    │   ╰── 0
    ├── 1
    │   ╰── 0
    ╰── 2
        ╰── 0


In [58]:
root.children[0].dictionary

{'a': -1.4291146798232646, 'b': 0.25831140138906844}

In [59]:
# STEP 1 cloning
root.rm_children()
[x.clone_children() for x in root.generation(0)]
[x.clone_children() for x in root.generation(1)]
[x.clone_children() for x in root.generation(2)];

In [60]:
for node in root.generation(1):
    node.mutate()
    node.run()

In [61]:
for node in root.generation(2):
    ryaml = ruamel.yaml.YAML()
        
    with open(node.parent.path+'/output.yaml', 'r') as file:
        parent_output = ryaml.load(file)
    with open(node.path+'/config.yaml', 'r') as file:
        cfg = ryaml.load(file)

    node.dictionary['sum_a_b']=parent_output['result']
    node.mutate()
    node.run()


In [62]:
for node in root.generation(3):
    ryaml = ruamel.yaml.YAML()
        
    with open(node.parent.path+'/output.yaml', 'r') as file:
        parent_output = ryaml.load(file)
    with open(node.path+'/config.yaml', 'r') as file:
        cfg = ryaml.load(file)

    node.dictionary['a']=parent_output['result']
    node.mutate()
    node.run()

In [63]:
my_list=[]
for node in root.generation(3):
    ryaml = ruamel.yaml.YAML()
    with open(node.path+'/output.yaml', 'r') as file:
        output = ryaml.load(file)
    my_list.append(output['result'])

In [64]:
my_list

[0.28569021676423856,
 0.9827845498991924,
 0.5206211484643671,
 0.23024329946287903,
 0.7920451739398001,
 0.41957870434012706,
 0.3043290135277088,
 1.0469026765026184,
 0.5545871410240621,
 0.4496245906111057,
 1.5467246513099964,
 0.8193632718440232,
 0.45029268018191837,
 1.5490229032962917,
 0.8205807498647026,
 0.10454025868427624,
 0.35962222382349424,
 0.1905065919071497,
 0.24402685226229887,
 0.8394610878881886,
 0.44469685213541926,
 0.33248643782913245,
 1.143765221821178,
 0.6058991906407387,
 0.3559960630702155,
 1.2246391723629408,
 0.6487414280530197,
 0.18395576663871963,
 0.6328144077351734,
 0.33522765875153177]

In [ ]:
[0.28569022 0.2302433  0.30432901 0.44962459 0.45029268 0.10454026
 0.24402685 0.33248644 0.35599606 0.18395577]
[0.98278455 0.79204517 1.04690268 1.54672465 1.5490229  0.35962222
 0.83946109 1.14376522 1.22463917 0.63281441]
[0.52062115 0.4195787  0.55458714 0.81936327 0.82058075 0.19050659
 0.44469685 0.60589919 0.64874143 0.33522766]